<a href="https://colab.research.google.com/github/HYAD-Yassin/Password_Factory/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 acoustique_voy_orales_20loc_ESTER_NCCFr_contexte_freqLex_distCentroide.csv
 AllDataSet_Filtred.txt
 AllDataSet.txt
'Alternance (1).gdoc'
 Alternance.gdoc
 archive.zip
 Ashley-Madison.txt
'Colab Notebooks'
 DATABASE_Password.zip
 data.zip
 Filtered-Ashley-Madison.txt
 Filtered_PWD.txt
 generated100k_GRU_passwords.txt
 generated1M_GRU_passwords.txt
 generated2_passwords.txt
 generated_GRU_passwords.txt
 generated_LSTM_passwords.txt
 generated_passwords.txt
 gru_model.pth
 histo2.png
 histogram1.png
 Letter.gdoc
 my_model
 nameGeneration.py
 Passwords.txt
 pwd_2Rnn.pth
 pwd_Rnn.pth
 __pycache__
 reduced_Ashley-Madison.txt
 rnn.pt
'Untitled document (1).gdoc'
'Untitled document.gdoc'


In [ ]:
from collections import defaultdict
import numpy as np

# Define the file path
file_path = '/content/drive/My Drive/Ashley-Madison.txt'

# Read the passwords
with open(file_path, 'r', encoding='utf-8') as file:
    passwords = file.read().splitlines()

# Group passwords by their first character
passwords_by_first_char = defaultdict(list)
for pwd in passwords:
    first_char = pwd[0]
    passwords_by_first_char[first_char].append(pwd)

# Calculate how many passwords to take from each group
desired_total = 60000  # Target number of passwords
groups_count = len(passwords_by_first_char)  # Number of different starting characters
passwords_per_group = desired_total // groups_count  # Evenly distribute the desired total across the groups

# Sample from each group
reduced_passwords = []
for char, pwd_list in passwords_by_first_char.items():
    # If there are fewer passwords than the target per group, take them all; otherwise, take a sample.
    if len(pwd_list) <= passwords_per_group:
        reduced_passwords.extend(pwd_list)
    else:
        reduced_passwords.extend(np.random.choice(pwd_list, passwords_per_group, replace=False))

# Output the reduced list of passwords
with open('/content/drive/My Drive/reduced_Ashley-Madison.txt', 'w') as file:
    for pwd in reduced_passwords:
        file.write(f"{pwd}\n")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import Counter
import os
from torch.utils.data import DataLoader, TensorDataset

# Check if CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_path = '/content/drive/My Drive/reduced_Ashley-Madison.txt'
#file_path = '/content/drive/My Drive/Ashley-Madison.txt'

# Read the data
with open(file_path, 'r', encoding='utf-8') as file:
    passwords = file.read().splitlines()  # Each password is a line

print(f"The total passwords in The DataSet is: {len(passwords)}")


# Preprocessing: Create a dictionary to convert characters to integers and back
all_chars = ''.join(set(''.join(passwords)))
n_characters = len(all_chars)
char_to_int = {char: i for i, char in enumerate(all_chars)}
int_to_char = {i: char for i, char in enumerate(all_chars)}

The total passwords in The DataSet is: 38839


In [ ]:
# GRU Model
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input.view(1, -1))
        output, hidden = self.gru(embedded, hidden)
        output = self.fc(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size).to(device)


In [ ]:
# Convert string to tensor
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = char_to_int[string[c]]
    return tensor
# Training function that uses the entire dataset
def train_on_full_dataset(passwords):
    total_loss = 0
    for password in passwords:
        input_line_tensor = char_tensor(password[:-1])
        target_line_tensor = char_tensor(password[1:])
        hidden = gru.init_hidden()

        gru.zero_grad()
        loss = 0

        for i in range(input_line_tensor.size(0)):
            output, hidden = gru(input_line_tensor[i].unsqueeze(0).to(device), hidden)
            l = criterion(output, target_line_tensor[i].unsqueeze(0).to(device))
            loss += l

        loss.backward()
        optimizer.step()
        total_loss += loss.item() / input_line_tensor.size(0)

    return total_loss / len(passwords)  # Return the average loss

# Initialize network, optimizer, and loss function as before
hidden_size = 128
n_layers = 2
gru = GRU(n_characters, hidden_size, n_characters, n_layers).to(device)
optimizer = optim.Adam(gru.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

# Training loop
n_epochs = 10  # Number of times you go through the entire dataset
for epoch in range(1, n_epochs + 1):
    loss = train_on_full_dataset(passwords)
    print(f'Epoch: {epoch} of {n_epochs}, Loss: {loss:.4f}')

Epoch: 1 of 10, Loss: 3.0577
Epoch: 2 of 10, Loss: 3.0562
Epoch: 3 of 10, Loss: 3.0516
Epoch: 4 of 10, Loss: 3.0489
Epoch: 5 of 10, Loss: 3.0479
Epoch: 6 of 10, Loss: 3.0482
Epoch: 7 of 10, Loss: 3.0459
Epoch: 8 of 10, Loss: 3.0451
Epoch: 9 of 10, Loss: 3.0424
Epoch: 10 of 10, Loss: 3.0436


In [ ]:
rnn = GRU(n_characters, hidden_size, n_characters).to(device)

#**Passwords Generation**

In [ ]:
import random
import torch


def generate_password(model, temperature, all_chars, char_to_int, device, int_to_char):
    model.eval()
    hidden = model.init_hidden()

    start_str = random.choice(all_chars)
    predict_len = random.randint(6, 15)

    input_tensor = char_tensor(start_str)
    predicted_str = start_str
    last_char = input_tensor[-1]

    for p in range(predict_len):
        output, hidden = model(last_char.unsqueeze(0).to(device), hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        # Clamp the output distribution to avoid 'inf' and 'nan'
        output_dist = torch.clamp(output_dist, min=0.0001, max=1.0)
        output_dist = output_dist / torch.sum(output_dist)
        top_char = torch.multinomial(output_dist, 1)[0]

        predicted_char = int_to_char[top_char.item()]
        predicted_str += predicted_char
        last_char = char_tensor(predicted_char).to(device)

    return predicted_str

def generate_passwords_to_file(model, num_pwd, file_path, temperature, all_chars, char_to_int, device, int_to_char):
    model.to(device)
    with open(file_path, 'w') as file:
        for _ in range(num_pwd):
            password = generate_password(model, temperature, all_chars, char_to_int, device, int_to_char)
            file.write(password + '\n')

# Example usage:
num_pwd = 1000000
output_file_path = '/content/drive/My Drive/generated1M_GRU_passwords.txt'  # Replace with your desired file path
generate_passwords_to_file(gru, num_pwd, output_file_path, 0.85, all_chars, char_to_int, device, int_to_char)


OSError: [Errno 107] Transport endpoint is not connected

#**Calculate Accuracy**

In [ ]:
def calculate_accuracy_and_matches(original_dataset_path, generated_file_path):
    # Load the original dataset
    with open(original_dataset_path, 'r') as file:
        original_passwords = set(file.read().splitlines())

    # Load generated passwords
    with open(generated_file_path, 'r') as file:
        generated_passwords = file.read().splitlines()

    # Find matches
    matches = [password for password in generated_passwords if password in original_passwords]
    dub = set(matches)
    # Calculate accuracy based on full line matches
    accuracy = (len(dub) / len(generated_passwords)) * 100 if generated_passwords else 0
    return accuracy, matches

# Paths to the files
original_dataset_path = '/content/drive/My Drive/Ashley-Madison.txt'  # Modify as needed
generated_file_path = '/content/drive/My Drive/generated1M_GRU_passwords.txt'  # Modify as needed

# Calculate accuracy and get matches
accuracy, matching_passwords = calculate_accuracy_and_matches(original_dataset_path, generated_file_path)
print(f"Accuracy: {accuracy:.2f}%")
print("Matching Passwords:", set(matching_passwords))

Accuracy: 0.00%
Matching Passwords: {'7123890'}


In [ ]:
torch.save(gru.state_dict(), '/content/drive/My Drive/gru_model.pth')
# Ensure your GRU model architecture is defined as `gru` here
gru.load_state_dict(torch.load('/content/drive/My Drive/gru_model.pth'))
gru.eval()  # Set the model to evaluation mode


GRU(
  (embedding): Embedding(90, 64)
  (gru): GRU(64, 64, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=64, out_features=90, bias=True)
)